In [2]:

import fastf1
import os

In [ ]:


#this function is going to be used to check what platform I'm running this on so I dont have to put the data file on github
cache_path = r'./dataFileDesktop'

# Create the cache directory if it doesn't exist
if not os.path.exists(cache_path):
    os.makedirs(cache_path)

if os.path.isfile(r'C:\Programming\SimplyLovelyF1\dataFileDesktop'):
    file = r'C:\Programming\SimplyLovelyF1\dataFileDesktop'
else:
    print("not on your desktop bro")


not on your desktop bro


In [9]:
import os
import fastf1
import pandas as pd
import time

# Set up cache path
cache_path = r'./dataFileDesktop'
if not os.path.exists(cache_path):
    os.makedirs(cache_path)

fastf1.Cache.enable_cache(cache_path)

# Years and sessions to include
years = [2021,2022]
practice_sessions = ['FP1', 'FP2', 'FP3']

all_laps = []

for year in years:
    try:
        schedule = fastf1.get_event_schedule(year, include_testing=False)
        for _, event in schedule.iterrows():
            location = event['EventName']
            for practice in practice_sessions:
                try:
                    session = fastf1.get_session(year, location, practice)
                    session.load()
                    laps = session.laps
                    if laps.empty:
                        print(f"No laps for {year} {location} {practice}")
                        continue

                    # Merge with weather data if available
                    if session.weather_data is not None and not session.weather_data.empty:
                        laps = laps.merge(session.weather_data, on='Time', how='left')
                    else:
                        laps['TrackTemperature'] = pd.NA
                        laps['Rainfall'] = pd.NA

                    # Add missing columns if not present
                    if 'TrackTemperature' not in laps.columns:
                        laps['TrackTemperature'] = pd.NA
                    if 'Rainfall' not in laps.columns:
                        laps['Rainfall'] = pd.NA

                    # Extract only necessary columns
                    subset = laps[['Driver', 'Team', 'LapNumber', 'LapTime',
                                   'TrackTemperature', 'Rainfall']].copy()
                    subset['Year'] = year
                    subset['Track'] = location

                    all_laps.append(subset)
                    print(f"✅ Added: {year} {location} {practice} - {len(subset)} laps")

                except Exception as e:
                    print(f"⚠️ Skipped {year} {location} {practice}: {e}")
                print("sleep 1")
                time.sleep(1)
    except Exception as e:
        print(f"⚠️ Skipped year {year}: {e}")

# Concatenate all collected lap data
if all_laps:
    df = pd.concat(all_laps, ignore_index=True)
    df = df[['Year', 'Track', 'Team', 'Driver', 'Rainfall', 'TrackTemperature', 'LapNumber', 'LapTime']]
    df.to_csv('practice_laps_2020_2023.csv', index=False)
    print("\n✅ Data saved to practice_laps_2020_2023.csv")
else:
    print("\n❌ No lap data was collected.")


logger      WARNING 	Failed to load schedule from FastF1 backend!
req            INFO 	No cached data found for season_schedule. Loading data...
_api           INFO 	Fetching season schedule...
logger      WARNING 	Failed to load schedule from F1 API backend!
logger      WARNING 	Failed to load schedule from Ergast API backend!


⚠️ Skipped year 2021: Failed to load any schedule data.


logger      WARNING 	Failed to load schedule from FastF1 backend!
req            INFO 	No cached data found for season_schedule. Loading data...
_api           INFO 	Fetching season schedule...
logger      WARNING 	Failed to load schedule from F1 API backend!
logger      WARNING 	Failed to load schedule from Ergast API backend!


⚠️ Skipped year 2022: Failed to load any schedule data.

❌ No lap data was collected.


In [14]:
try:
    schedule = fastf1.get_event_schedule(2021)
    print(schedule.head())
except Exception as e:
    print(f"Failed to fetch 2023 schedule: {e}")

logger      WARNING 	Failed to load schedule from FastF1 backend!
req            INFO 	No cached data found for season_schedule. Loading data...
_api           INFO 	Fetching season schedule...
logger      WARNING 	Failed to load schedule from F1 API backend!
logger      WARNING 	Failed to load schedule from Ergast API backend!


Failed to fetch 2023 schedule: Failed to load any schedule data.
